# Note:
#### This notebook is a demo of some SparkSQL commands
### train.csv and test.csv


# Run cells below to setup spark environment

In [ ]:
!pip install pyspark

### Mounting this notebook with google drive "/contesnt/gdrive/" is the path to google drive




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Import necessary libararies


*   SparkSession is used to create spark dataframe
*   import functions as f then when you want to use function you just simply call it by "f" instead of typing the full word "functions"
* functions module contain many builtin function such as concatenate, add_date etc.
* UserDefinedFucntion is a module that allow you to create your on function  that will be applied to each entry of dataframe



In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import UserDefinedFunction
from datetime import date, datetime, timedelta
from pyspark.sql.types import *

### Creating SparkSession object
Spark session objects can be created by using SparkSession.builder.getorCreated(). You also have to specify APP_NAME for your sparkSession that you want to work on.


In [ ]:
from pyspark.sql import SparkSession
APP_NAME = "Covid19_demo"
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
spark

### Creating a Dataframe from csv files
You can read csv files from google drive by providing the path “/gdrive/My Drive/”. This path is the path to access files in your google drive. Since we’ve put dataset in “data” folder, the path name should be “/gdrive/My Drive/data/filename.csv”

You can create a spark dataframe by calling sparkSession object name “spark” that you have created in the previous step. 
spark.read(‘csv’): specify that you want spark to read .csv file.
option(‘header’,’true’): contain header to the dataframe
option(‘inferSchema’, ‘true’): ensure that we detect the correct type for each column. Without this option, the default type will be string for all columns.
 printSchema to see the Schema of data


In [ ]:
#To see the files existing in the current directory of the data folder
import os
os.listdir('./gdrive/My Drive/Covid19')

['Covid19_test.csv',
 'Covid19_train.csv',
 'data',
 'Machine learning spark',
 'Spark preprocessing assignment',
 'dataPreprocessingSparkSQLdemo_v2.ipynb',
 'dataPreprocessingSparkSQLdemo.ipynb',
 'Covid19 pgm.ipynb',
 'Covid19_demo.ipynb',
 'Covid19_demo_prof.ipynb',
 'Covid19_demo_v3.ipynb',
 'Covid19_demo_v2.ipynb']

In [ ]:
#Create DF called train
#file_folder = './gdrive/My Drive/data/'
file_folder = "./gdrive/My Drive/Covid19/data/"
A_DF = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(file_folder + 'A.csv')
A_DF.show()
# null values in csv file is marked as "null" without quotes.

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province/State|Country/Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|          null|   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|          null|   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|          null|   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|          null|   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|          null|   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|          null|   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|          null|   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|          null|   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|          null|   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

### Rename Columns
We can rename columns of dataframe by directly calling command withColumnRenamed() from a data frame object. In this case “train” is a dataFrame object.
* withColumnRenamed(): to rename columns result in a new data frame because the dataframe consists of Row RDDs. They are immutable
* We have to assign it back to “train” (train = train.withColumnRenamed()) in order to modify the dataframe
* In order to see content in spark dataframe, the command .show() is called directly from a dataframe object
* .show() command will show the first 20 rows of the dataframe

In [ ]:
# rename column into a new dataframe.
#withColumnRenamed is a Spark method
A2_DF = A_DF.withColumnRenamed('Province/State','Province_State').withColumnRenamed('Country/Region','Country_Region')
A2_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|          null|   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|          null|   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|          null|   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|          null|   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|          null|   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|          null|   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|          null|   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|          null|   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|          null|   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

The following cell shows the examples of creating variables as a list which can store the column names.

In [ ]:
# crete variables that store some column names which will be used later.
#ynames and cp are two variables that contain two columns
#ny = len(ynames) will store #rows
#cpd is a new dataframe with 3 columns
ynames = ['ConfirmedCases', 'Fatalities']
ny = len(ynames)
cp = ['Country_Region','Province_State']
cpd = cp + ['Date']
cpd

['Country_Region', 'Province_State', 'Date']

### Fill missing values with empty string
An example of filling missing value with empty string in "Provinve_State" column

In [ ]:
# filling missing values with empty string with method called "fillna"
# fillna replaces "null" with an empty string
A3_DF = A2_DF.fillna({'Province_State' : '' })
#train = train.fillna({'Province_State', 0})
A3_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|
+---+--------------+--------------+----+----+----------+--------------+----------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|
|  7|              |   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|
|  8|              |   Afghanistan|33.0|65.0|2020-01-29|           0.0|       0.0|
|  9|              |   Afghanistan|33.0|65.0|2020-01-30|           0.0|       0.0|
| 10

### Create new column from existing columns
* withColumn() can also create a new column as well
* by specify column name, if the column name doesn’t exist in the dataframe, it will automatically create a new column with the name
* In this case “PSCR” is the column name. 
* after that, build in function f.concat() is call to concat value in two columns with “_”
build in function f.lit() is necessary. It is use to pass a constant or value to operate with value in a dataframe i.e. in this case, “_’ is passed into f.lit() fiction in order to let it connect between two fields value
* For example, for a tuple “province_state = Busan”, “Country_regoin = South Korea” the result of will be “Busan_South Korea”


In [ ]:
# Example of creating a new column "PSCR" by concatinating two columns connected by "_"
#lit() PySpark SQL functions lit() and typedLit() are used to add a new column to DataFrame by assigning a literal or constant value. Both these functions return Column type as return type.
A4_DF = A3_DF.withColumn('PSCR',f.concat(f.col('Province_State'),f.lit('_'),f.col('Country_Region')))
A4_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|_Afghanistan|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|_Afghanistan|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|_Afghanistan|
|  7|              |   Afghanistan|33.0|65.0|2020-01-28|           0.0|       0.0|_Afghanistan|
|  8|              |   Afghanistan|33.0|

In [ ]:
A4_DF.count()

17892

### Repeat the same procedure for the dataset A (A.csv) to dataset B (B.csv)
* rename columns
* filling missing value empty string
* modify existing column
* f.concat() and f.lit()


Preprocess the data on the test data.

In [ ]:
#file_folder = "./gdrive/My Drive/Covid19/data/"
B_DF = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(file_folder + 'B.csv')
# rename columns
B2_DF = B_DF.withColumnRenamed('Province/State','Province_State').withColumnRenamed('Country/Region','Country_Region')
# filling missing values with empty string
B3_DF = B2_DF.fillna({'Province_State' : ''})
# create a new column "key" by concatinating two columns
B4_DF = B3_DF.withColumn('PSCR',f.concat(f.col('Province_State'),f.lit('_'),f.col('Country_Region')))
B4_DF.show()

+----------+--------------+--------------+----+----+----------+------------+
|ForecastId|Province_State|Country_Region| Lat|Long|      Date|        PSCR|
+----------+--------------+--------------+----+----+----------+------------+
|         1|              |   Afghanistan|33.0|65.0|2020-03-12|_Afghanistan|
|         2|              |   Afghanistan|33.0|65.0|2020-03-13|_Afghanistan|
|         3|              |   Afghanistan|33.0|65.0|2020-03-14|_Afghanistan|
|         4|              |   Afghanistan|33.0|65.0|2020-03-15|_Afghanistan|
|         5|              |   Afghanistan|33.0|65.0|2020-03-16|_Afghanistan|
|         6|              |   Afghanistan|33.0|65.0|2020-03-17|_Afghanistan|
|         7|              |   Afghanistan|33.0|65.0|2020-03-18|_Afghanistan|
|         8|              |   Afghanistan|33.0|65.0|2020-03-19|_Afghanistan|
|         9|              |   Afghanistan|33.0|65.0|2020-03-20|_Afghanistan|
|        10|              |   Afghanistan|33.0|65.0|2020-03-21|_Afghanistan|

In [ ]:

B4_DF.count()

12212

### Aggreagte funtion (max)

To show the last day from the training dataset.

In [ ]:
# tmax is the last day of training; Either syntax below is working

#tmax = train.agg({'Date':'max'}).show()

+----------+
| max(Date)|
+----------+
|2020-03-24|
+----------+



Collect (Action) - Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data.

####collect()[0]:  Row(max(Date)='2020-03-24')
####collect(): [Row(max(Date)='2020-03-24')]

In [ ]:
tmax = A4_DF.agg({'Date':'max'}).collect()[0]['max(Date)']
#tmax = A4_DF.agg({'Date':'max'}).collect()[0]
#tmax = A4_DF.agg({'Date':'max'}).collect()
tmax

'2020-03-24'

Convert the string to a datetime object
print()

In [ ]:
print(type(tmax))
tmax=datetime.strptime(tmax, '%Y-%m-%d').date()
print(tmax,type(tmax))

<class 'str'>
2020-03-24 <class 'datetime.date'>


The same example on the test data

In [ ]:
fmax = B4_DF.agg({'Date':'max'}).show()

+----------+
| max(Date)|
+----------+
|2020-04-23|
+----------+



In [ ]:
fmax = B4_DF.agg({'Date':'max'}).collect()[0]['max(Date)']
fmax

'2020-04-23'

In [ ]:
print(type(fmax))
fmax=datetime.strptime(fmax, '%Y-%m-%d').date()
print(fmax,type(fmax))

<class 'str'>
2020-04-23 <class 'datetime.date'>


### Aggregate function (min)

An example of using min aggregate function and collect the value

### type(tmin) below is a string that is selected from the result of collect()[0]['min(Date)']

In [ ]:
tmin = A4_DF.agg({'Date':'min'}).collect()[0]['min(Date)']
fmin = B4_DF.agg({'Date':'min'}).collect()[0]['min(Date)']
#tmin,fmin
print(tmin)
print(fmin)

2020-01-22
2020-03-12


In [ ]:
tmin=datetime.strptime(tmin, '%Y-%m-%d').date()
fmin=datetime.strptime(fmin, '%Y-%m-%d').date()
print(tmin)
print(fmin)

2020-01-22
2020-03-12


### Using SQL statement to exact column and merge dataframes
We can use SQL command in Spark SQL by 
* In order to use SQL statement, we should first create a view of a DF using command creatieOrReplaceTempView(“viewname”) 
* In this case, we create view name “train” from train dataframe
* view name “test” from test data frame
* We use command spark.sql(“SQL statement)
* The result is a new dataframe
* From example below first SQL statement is used to create a new dataframe from ‘test
* “SELECT Country_Region, Province_State, ForecastId, Date FROM test” the new dataframe from this query is assigned to “test_sub_col”
* then view name “sub_test” is created from test_sub_col dataframe
* then the last SQL statement is used to merge two view together e.g. “train” and “sub_test”
* the result is assigned to merge_train 


In [ ]:
# join two data frames. In order to use spark.sql, you have to create view first and use it inside spark.sql
A4_DF.createOrReplaceTempView('A4_V')
A5_DF = spark.sql('select * from A4_V')
A5_DF.show(3)
A5_DF.count()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+
only showing top 3 rows



17892

In [ ]:
B4_DF.createOrReplaceTempView('B4_V')
B5_DF = spark.sql('select * from B4_V')
B5_DF.show(3)
B5_DF.count()

+----------+--------------+--------------+----+----+----------+------------+
|ForecastId|Province_State|Country_Region| Lat|Long|      Date|        PSCR|
+----------+--------------+--------------+----+----+----------+------------+
|         1|              |   Afghanistan|33.0|65.0|2020-03-12|_Afghanistan|
|         2|              |   Afghanistan|33.0|65.0|2020-03-13|_Afghanistan|
|         3|              |   Afghanistan|33.0|65.0|2020-03-14|_Afghanistan|
+----------+--------------+--------------+----+----+----------+------------+
only showing top 3 rows



12212

In [ ]:

B_sub_col_DF = spark.sql('SELECT Country_Region, Province_State, ForecastId, Date FROM B4_V')
B_sub_col_DF.show(3)
B_sub_col_DF.count()

+--------------+--------------+----------+----------+
|Country_Region|Province_State|ForecastId|      Date|
+--------------+--------------+----------+----------+
|   Afghanistan|              |         1|2020-03-12|
|   Afghanistan|              |         2|2020-03-13|
|   Afghanistan|              |         3|2020-03-14|
+--------------+--------------+----------+----------+
only showing top 3 rows



12212

In [ ]:
# Dataset A does not  have ForecaseID, but dataset B does. So, add it to the merged dataset.
# ForecastID will be used in ML part later.
# Note the use of the backslash at the end of the line to continue in the nextline
B_sub_col_DF.createOrReplaceTempView('B_sub_col_V')
merge_AB_DF = spark.sql('SELECT t.*, s.ForecastId FROM A4_V t LEFT JOIN B_sub_col_V s \
                      ON t.Province_State = s.Province_State\
                      AND t.Country_Region = s.Country_Region\
                      AND t.Date = s.Date')
merge_AB_DF.show()
merge_AB_DF.count()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|ForecastId|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|      null|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|      null|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|      null|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|_Afghanistan|      null|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|_Afghanistan|      null|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|_Afghanistan|      null|
|  7|              |   Afghanistan|33

17892

### Another example of
* filling missing value with 0

In [ ]:
merge_AB2_DF = merge_AB_DF.fillna({"ForecastId" : 0})
merge_AB2_DF.show()
print(merge_AB2_DF.count())

+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|ForecastId|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|         0|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|         0|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|         0|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|_Afghanistan|         0|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|_Afghanistan|         0|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|_Afghanistan|         0|
|  7|              |   Afghanistan|33

### Another example of
* using SQL statement for spark dataframe

In [ ]:
# display bottom of the data
merge_AB2_DF.createOrReplaceTempView('merge_AB2_V')
spark.sql('SELECT * FROM merge_AB2_V ORDER BY Id DESC LIMIT 20').show()

+-----+--------------+--------------+--------+-------+----------+--------------+----------+-------+----------+
|   Id|Province_State|Country_Region|     Lat|   Long|      Date|ConfirmedCases|Fatalities|   PSCR|ForecastId|
+-----+--------------+--------------+--------+-------+----------+--------------+----------+-------+----------+
|26382|              |        Zambia|-15.4167|28.2833|2020-03-24|           3.0|       0.0|_Zambia|     12182|
|26381|              |        Zambia|-15.4167|28.2833|2020-03-23|           3.0|       0.0|_Zambia|     12181|
|26380|              |        Zambia|-15.4167|28.2833|2020-03-22|           3.0|       0.0|_Zambia|     12180|
|26379|              |        Zambia|-15.4167|28.2833|2020-03-21|           2.0|       0.0|_Zambia|     12179|
|26378|              |        Zambia|-15.4167|28.2833|2020-03-20|           2.0|       0.0|_Zambia|     12178|
|26377|              |        Zambia|-15.4167|28.2833|2020-03-19|           2.0|       0.0|_Zambia|     12177|
|

### Another example of
* create new columns for prediction variables
* use f.lit() to adding value to a column
nan: NaN
* NaN , standing for **not a number**, is a numeric data type used to represent any value that is undefined or unpresentable. For example, 0/0 is undefined as a real number and is, therefore, represented by NaN.

In [ ]:
#y0_prd and y1_prd will be used in ML algorithms later
C_DF = merge_AB2_DF
C2_DF =C_DF.withColumn('y0_prd',f.lit(float('nan'))).withColumn('y1_prd',f.lit(float('nan')))
#train =C2_DF.withColumn('y1_prd',f.lit(float('nan')))
C2_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+------+------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|ForecastId|y0_prd|y1_prd|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+------+------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  6|              |   Afghanist

Display the data with LIMIT keyword

In [ ]:
C2_DF.createOrReplaceTempView('C2_V')
spark.sql('SELECT * FROM C2_V ORDER BY Id DESC LIMIT 20').show(5)

+-----+--------------+--------------+--------+-------+----------+--------------+----------+-------+----------+------+------+
|   Id|Province_State|Country_Region|     Lat|   Long|      Date|ConfirmedCases|Fatalities|   PSCR|ForecastId|y0_prd|y1_prd|
+-----+--------------+--------------+--------+-------+----------+--------------+----------+-------+----------+------+------+
|26382|              |        Zambia|-15.4167|28.2833|2020-03-24|           3.0|       0.0|_Zambia|     12182|   NaN|   NaN|
|26381|              |        Zambia|-15.4167|28.2833|2020-03-23|           3.0|       0.0|_Zambia|     12181|   NaN|   NaN|
|26380|              |        Zambia|-15.4167|28.2833|2020-03-22|           3.0|       0.0|_Zambia|     12180|   NaN|   NaN|
|26379|              |        Zambia|-15.4167|28.2833|2020-03-21|           2.0|       0.0|_Zambia|     12179|   NaN|   NaN|
|26378|              |        Zambia|-15.4167|28.2833|2020-03-20|           2.0|       0.0|_Zambia|     12178|   NaN|   NaN|


Aggregate function (groupby and count)
An example of using aggregate function with group by and count

In [ ]:
C2_DF.groupby('Country_region').count().show()

+--------------+-----+
|Country_region|count|
+--------------+-----+
|      Paraguay|   63|
|        Russia|   63|
|       Senegal|   63|
|        Sweden|   63|
|        Guyana|   63|
|        Jersey|   63|
|   Philippines|   63|
|      Djibouti|   63|
|      Malaysia|   63|
|     Singapore|   63|
|        Turkey|   63|
|          Iraq|   63|
|       Germany|   63|
|   Afghanistan|   63|
|      Cambodia|   63|
|        Jordan|   63|
|      Maldives|   63|
|        Rwanda|   63|
|         Sudan|   63|
|        France|  504|
+--------------+-----+
only showing top 20 rows



In [ ]:
#Calculate the maxid inorder to substitute in the test dataframe instead on null values in the ID column
max_id=C2_DF.agg({'ID':'max'}).collect()[0]['max(ID)']
max_id

26382

In [ ]:
B6_DF = B5_DF.withColumn('Id',f.lit(max_id)+f.col('ForecastID'))
B6_DF.show()

+----------+--------------+--------------+----+----+----------+------------+-----+
|ForecastId|Province_State|Country_Region| Lat|Long|      Date|        PSCR|   Id|
+----------+--------------+--------------+----+----+----------+------------+-----+
|         1|              |   Afghanistan|33.0|65.0|2020-03-12|_Afghanistan|26383|
|         2|              |   Afghanistan|33.0|65.0|2020-03-13|_Afghanistan|26384|
|         3|              |   Afghanistan|33.0|65.0|2020-03-14|_Afghanistan|26385|
|         4|              |   Afghanistan|33.0|65.0|2020-03-15|_Afghanistan|26386|
|         5|              |   Afghanistan|33.0|65.0|2020-03-16|_Afghanistan|26387|
|         6|              |   Afghanistan|33.0|65.0|2020-03-17|_Afghanistan|26388|
|         7|              |   Afghanistan|33.0|65.0|2020-03-18|_Afghanistan|26389|
|         8|              |   Afghanistan|33.0|65.0|2020-03-19|_Afghanistan|26390|
|         9|              |   Afghanistan|33.0|65.0|2020-03-20|_Afghanistan|26391|
|   

### Another example of 
* using max

Calculate the last day of training to filter out the date later than the last day of train from the test dataframe. The filtering is been shown in the next code cell.

In [ ]:
C2_date_max = C2_DF.agg({'Date':'max'}).show()
C2_date_max

+----------+
| max(Date)|
+----------+
|2020-03-24|
+----------+



# Merging Data Frames (unionByName)
* use filter to filter only date need and assign to a new dataframe
* unionByName() will join data by row instead of column


In [ ]:
B6_DF.printSchema()

root
 |-- ForecastId: integer (nullable = true)
 |-- Province_State: string (nullable = false)
 |-- Country_Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- PSCR: string (nullable = true)
 |-- Id: integer (nullable = true)



In [ ]:
B7_DF =B6_DF.withColumn('y0_prd',f.lit(float('0'))).withColumn('y1_prd',f.lit(float('0')))
B8_DF =B7_DF.withColumn('ConfirmedCases',f.lit(float('nan'))).withColumn('Fatalities',f.lit(float('nan')))
B8_DF.show()

+----------+--------------+--------------+----+----+----------+------------+-----+------+------+--------------+----------+
|ForecastId|Province_State|Country_Region| Lat|Long|      Date|        PSCR|   Id|y0_prd|y1_prd|ConfirmedCases|Fatalities|
+----------+--------------+--------------+----+----+----------+------------+-----+------+------+--------------+----------+
|         1|              |   Afghanistan|33.0|65.0|2020-03-12|_Afghanistan|26383|   0.0|   0.0|           NaN|       NaN|
|         2|              |   Afghanistan|33.0|65.0|2020-03-13|_Afghanistan|26384|   0.0|   0.0|           NaN|       NaN|
|         3|              |   Afghanistan|33.0|65.0|2020-03-14|_Afghanistan|26385|   0.0|   0.0|           NaN|       NaN|
|         4|              |   Afghanistan|33.0|65.0|2020-03-15|_Afghanistan|26386|   0.0|   0.0|           NaN|       NaN|
|         5|              |   Afghanistan|33.0|65.0|2020-03-16|_Afghanistan|26387|   0.0|   0.0|           NaN|       NaN|
|         6|    

In [ ]:
# the new DF contains rows whose date is greater than the max date of Train data
print('#rows in C2 = ', C2_DF.count())
test_no_overlap = B8_DF.filter(B8_DF['Date'] > C2_date_max)
print ('#rows in test_no_overlap = ', test_no_overlap.count())
test_no_overlap.printSchema()

#rows in C2 =  17892
#rows in test_no_overlap =  0
root
 |-- ForecastId: integer (nullable = true)
 |-- Province_State: string (nullable = false)
 |-- Country_Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- PSCR: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- y0_prd: double (nullable = false)
 |-- y1_prd: double (nullable = false)
 |-- ConfirmedCases: double (nullable = false)
 |-- Fatalities: double (nullable = false)



In [ ]:
C2_DF.printSchema()


root
 |-- Id: integer (nullable = true)
 |-- Province_State: string (nullable = false)
 |-- Country_Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- ConfirmedCases: double (nullable = true)
 |-- Fatalities: double (nullable = true)
 |-- PSCR: string (nullable = true)
 |-- ForecastId: integer (nullable = false)
 |-- y0_prd: double (nullable = false)
 |-- y1_prd: double (nullable = false)



In [ ]:
D_DF = C2_DF.unionByName(test_no_overlap)
D_DF.show(5)
print(D_DF.count())

+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+------+------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|        PSCR|ForecastId|y0_prd|y1_prd|
+---+--------------+--------------+----+----+----------+--------------+----------+------------+----------+------+------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|_Afghanistan|         0|   NaN|   NaN|
+---+--------------+------------

In [ ]:
#Drop PSCR column
D2_DF = D_DF.drop("PSCR")
D2_DF.show()

+---+--------------+--------------+----+----+----------+--------------+----------+----------+------+------+
| Id|Province_State|Country_Region| Lat|Long|      Date|ConfirmedCases|Fatalities|ForecastId|y0_prd|y1_prd|
+---+--------------+--------------+----+----+----------+--------------+----------+----------+------+------+
|  1|              |   Afghanistan|33.0|65.0|2020-01-22|           0.0|       0.0|         0|   NaN|   NaN|
|  2|              |   Afghanistan|33.0|65.0|2020-01-23|           0.0|       0.0|         0|   NaN|   NaN|
|  3|              |   Afghanistan|33.0|65.0|2020-01-24|           0.0|       0.0|         0|   NaN|   NaN|
|  4|              |   Afghanistan|33.0|65.0|2020-01-25|           0.0|       0.0|         0|   NaN|   NaN|
|  5|              |   Afghanistan|33.0|65.0|2020-01-26|           0.0|       0.0|         0|   NaN|   NaN|
|  6|              |   Afghanistan|33.0|65.0|2020-01-27|           0.0|       0.0|         0|   NaN|   NaN|
|  7|              |   Afgha

In [ ]:
D2_DF.write.save(file_folder + 'D_merged_data3.csv')
os.listdir(file_folder)

['wmc.csv',
 'google_trends.csv',
 'covid19countryinfo2.csv',
 'states_daily_4pm_et.csv',
 'B.csv',
 'A.csv',
 'merged_data.csv',
 'merged_data2.csv',
 'D_merged_data3.csv']